In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import namedtuple

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import GridSearchCV

%matplotlib inline
sns.set_style('darkgrid')
seed = 42

# APS Failure at Scania Trucks

It seems like a lot of the classification tasks worth pursuing have low (< 5%) target prevalence, and in many of those tasks, there are a large number of both categorical and continuous predictors. In this notebook, I'll walk through a variety of approaches for dealing with unbalanced datasets.

## Data Description
`aps_failure_test_set.csv`: 11.9MB (16,000 obs)

`aps_failure_training_set.csv`: 44.7MB (60,000 obs)

The datasets' positive class consists of component failures for a specific component of the APS system. The negative class consists of trucks with failures for components not related to the APS.

The attributes are as follows: class, then anonymized operational data. The operational data have an identifier and a bin id, like `Identifier_Bin`. In total there are 171 attributes, of which 7 are histogram variables. Missing values are denoted by `na`.

## Challenge Metric
Since this dataset was part of a challenge, they also provided a "challenge metric" formula to weight the cost of false positives and false negatives:

`Cost_1(FP) = 10` and `cost_2(FN) = 500`

We will want to minimize this.

In [2]:
def cost(y_true, y_pred, fp_cost=10, fn_cost=500, normalize=True):
    cm = confusion_matrix(y_true, y_pred)
    fp = cm[1][1]
    fn = cm[0][1]
    
    c = fp * fp_cost + fn * fn_cost
    
    return c / len(y_true) if normalize else c

## Exploration

In [3]:
train_df = pd.read_csv('aps_failure_training_set.csv', header=14, na_values='na')
train_df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,neg,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,neg,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,neg,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,neg,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


Convert target into a binary variable and rename column so it doesn't use a keyword (aka `class`) that prevents dot accessibility.

In [4]:
train_df['target'] = train_df['class'].map({'neg': 0, 'pos': 1})
train_df = train_df.drop('class', axis=1)

## Metadata Generation
I find it helpful to put together a metadata-set that describes important characteristics of each variable.

In [5]:
def generate_metadata(df):
    meta = df.isnull().sum().to_frame('n_missing')
    meta['perc_missing'] = meta['n_missing'] / len(df)
    meta['n_unique'] = df.nunique()
    
    descs = train_df.describe().T
    descs['n_valid'] = descs['count'].copy()
    return meta.join(descs.drop('count', axis=1))

In [6]:
meta = generate_metadata(train_df)

Uncomment the cell below to view the metadata in its entirety.

In [7]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(meta.sort_values('perc_missing', ascending=True))

A few things to notice about `meta`...

- Everything is numeric. This is typical of UCI datasets, but normally, we would have to think about how to handle other types.
- `cd_000` has only one unique value. We'll start by encoding it as a binary variable.

For the baseline model, we will drop predictors that are over 25% missing and impute the median for the rest.

### Remove variables with > 25% missingness

In [8]:
bad_vars = meta[meta.perc_missing > 0.25].index

In [9]:
train_df = train_df.loc[:, ~train_df.columns.isin(bad_vars)]

### Train/Dev Split
We want to save the test set for an unbiased, out-of-sample assessment of the final model, so let's split the training data into a new training set and a development set. 

In [10]:
Data = namedtuple('Data', ['X_train', 'X_dev', 'y_train', 'y_dev'])

In [62]:
data = Data(*train_test_split(train_df.drop('target', axis=1), train_df.target, test_size=.2, stratify=train_df.target, random_state=seed))

In [133]:
Data = namedtuple('Data', ['X_train', 'y_train'])
data = Data(train_df.drop('target', axis=1), train_df.target)

## Establishing a baseline model

Let's start by getting a baseline model with logistic regression with L1 and L2 regularization (default in `sklearn`). 

In [90]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('lr', LogisticRegression(solver='lbfgs', random_state=seed))
])

In [111]:
def assess_model(pl, data):
    
    skf = StratifiedKFold(n_splits=5)
    costs = []
    for i, (train_idx, test_idx) in enumerate(skf.split(data.X_train, data.y_train)):
        X_train, y_train = data.X_train.iloc[train_idx], data.y_train.iloc[train_idx]
        X_test, y_test = data.X_train.iloc[test_idx], data.y_train.iloc[test_idx]
        pl.fit(X_train, y_train)
        y_preds = pl.predict(X_test)
        costs.append(cost(y_test, y_preds))
    print(np.mean(costs))
    
    # train assessment
#     y_preds = pl.predict(data.X_train)
#     print('##### Train #####')
#     print(classification_report(data.y_train, y_preds))
#     print(f'Normalized train cost: {cost(data.y_train, y_preds):.{2}f}\n')
    
    # dev assessment
#     y_preds = pl.predict(data.X_dev)
#     print('##### Test #####')
#     print(classification_report(data.y_dev, y_preds))
#     print(f'Normalized dev cost: {cost(data.y_dev, y_preds):.{2}f}\n')
    
    return pl

In [104]:
pl = assess_model(pl, data)

/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[1.8802083333333333, 1.5645833333333334, 2.1375, 1.4625, 0.940625]


/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


It looks like our baseline normalized cost for on the `dev` set is 1.44. Let's see if we can beat it!

## Experiment 1: ElasticNet
I'm always confused about the difference between elastic net and logistic regression in `sklearn` because the logistic regression uses L1 and L2 regularization by default.

### Vanilla ElasticNet

In [112]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='elasticnet', random_state=seed))
])

In [113]:
pl = assess_model(pl, data)

1.8672916666666666


### ElasticNet + Adaptive Learning Rate

In [114]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='elasticnet', learning_rate='adaptive', eta0=0.1, random_state=seed))
])

In [115]:
pl = assess_model(pl, data)

1.798125


### L1 Regularization

In [116]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='l1', random_state=seed))
])

In [117]:
pl = assess_model(pl, data)

1.8579166666666667


### L1 Regularization + Adaptive Learning Rate

In [118]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='l1', learning_rate='adaptive', eta0=1, random_state=seed))
])

In [119]:
pl = assess_model(pl, data)

2.0727083333333334


Interesting....the L1-only model gives very similar results for both the train and dev sets.

### L2 Penality only

In [120]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='l2', random_state=seed))
])

In [121]:
pl = assess_model(pl, data)

1.6983333333333335


### L2 Penality + Adaptive Learning Rate

In [122]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='l2', learning_rate='adaptive', eta0=1, random_state=seed))
])

In [123]:
pl = assess_model(pl, data)

1.9427083333333333


### No regularization + Adaptive Learning Rate

In [124]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='none', learning_rate='adaptive', eta0=1, random_state=seed))
])

In [125]:
pl = assess_model(pl, data)

2.382916666666667


### Results
The elastic net model with learning rate adaptation significantly outperformed the baseline model based on the challenge metric. However, I am having some heartburn about the whole challenge metric optimization...is a false  positive really 50 times worse than a false negative???

## Experiment 2: RandomForest

### Vanilla RandomForest

In [134]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=seed))
])

In [135]:
pl = assess_model(pl, data)

0.8146666666666667


Hmmm...this is the largest disparity in train/test cost that we've seen so far, which makes me think the model is overfit. I wonder if a RF would do better if we did some feature selection first.

### Feature Selection + Random Forest

In [128]:
from sklearn.feature_selection import SelectFromModel

In [129]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('selector', SelectFromModel(SGDClassifier(loss='log', penalty='elasticnet', learning_rate='adaptive', eta0=1, random_state=seed))),
    ('rf', RandomForestClassifier(random_state=seed))
])

In [130]:
pl = assess_model(pl, data)

/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9629166666666666


## Experiment 3: Predictive Imputation
Let's see if we can get any lift out of the new `IterativeImputer` from `sklearn`.

In [ ]:
pl = Pipeline([
    ('imputer', IterativeImputer(ExtraTreesRegressor(random_state=seed, n_estimators=10), random_state=seed)),
    ('scaler',  StandardScaler()),
    ('rf', RandomForestClassifier())
])

In [ ]:
pl = assess_model(pl, data)

Ran out of memory :(

## Experiment 4: ElasticNet Parameter Tuning

In [15]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='elasticnet', learning_rate='adaptive', eta0=1, random_state=seed))
])

In [34]:
np.logspace(-1, 1, 3)

array([ 0.1,  1. , 10. ])

In [38]:
param_grid = {
    'en__eta0': np.logspace(-1, 1, 3),
    'en__alpha': np.logspace(-1, 1, 3),
    'en__l1_ratio': np.random.random_sample(3)
}

In [39]:
grid = GridSearchCV(pl, param_grid, n_jobs=-1, verbose=1)

In [40]:
grid.fit(data.X_train, data.y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 15.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('imputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='median',
                                                      verbose=0)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('en',
                                        SGDClassifier(alpha=0.0001,
                                                 

In [44]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_en__alpha,param_en__eta0,param_en__l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,7.802815,0.162494,0.153891,0.020452,0.1,0.1,0.59064,"{'en__alpha': 0.1, 'en__eta0': 0.1, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
1,6.938443,0.057334,0.125233,0.019003,0.1,0.1,0.751729,"{'en__alpha': 0.1, 'en__eta0': 0.1, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
2,6.847812,0.057905,0.124929,0.017362,0.1,0.1,0.840148,"{'en__alpha': 0.1, 'en__eta0': 0.1, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
3,8.386170,0.154293,0.105901,0.005236,0.1,1,0.59064,"{'en__alpha': 0.1, 'en__eta0': 1.0, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
4,7.302342,0.060104,0.106760,0.021663,0.1,1,0.751729,"{'en__alpha': 0.1, 'en__eta0': 1.0, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
5,7.069584,0.017117,0.107690,0.021227,0.1,1,0.840148,"{'en__alpha': 0.1, 'en__eta0': 1.0, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1
6,9.308473,0.402296,0.114809,0.026464,0.1,10,0.59064,"{'en__alpha': 0.1, 'en__eta0': 10.0, 'en__l1_r...",0.983314,0.983313,0.983374,0.983333,0.000029,1
7,8.217208,0.064072,0.107169,0.016717,0.1,10,0.751729,"{'en__alpha': 0.1, 'en__eta0': 10.0, 'en__l1_r...",0.983314,0.983313,0.983374,0.983333,0.000029,1
8,103.532825,0.457535,0.372566,0.105444,0.1,10,0.840148,"{'en__alpha': 0.1, 'en__eta0': 10.0, 'en__l1_r...",0.983314,0.983313,0.983374,0.983333,0.000029,1
9,101.829350,0.031409,0.317255,0.008041,1,0.1,0.59064,"{'en__alpha': 1.0, 'en__eta0': 0.1, 'en__l1_ra...",0.983314,0.983313,0.983374,0.983333,0.000029,1


In [49]:
pl = Pipeline([
    ('imputer', SimpleImputer(np.nan, strategy='median')),
    ('scaler',  StandardScaler()),
    ('en', SGDClassifier(loss='log', penalty='elasticnet', learning_rate='adaptive', eta0=1, alpha=1, random_state=seed))
])

In [50]:
pl = assess_model(pl, data)

##### Train #####
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     47200
           1       0.00      0.00      0.00       800

    accuracy                           0.98     48000
   macro avg       0.49      0.50      0.50     48000
weighted avg       0.97      0.98      0.98     48000

Normalized train cost: 0.00

##### Test #####
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     11800
           1       0.00      0.00      0.00       200

    accuracy                           0.98     12000
   macro avg       0.49      0.50      0.50     12000
weighted avg       0.97      0.98      0.98     12000

Normalized dev cost: 0.00



/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/anaconda3/envs/scania/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
